In [1]:
import os
import google.cloud.storage
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import re
from collections import Counter
import csv
from nltk.tokenize import TweetTokenizer
import spacy
import datetime

# Download spacy
# python -m spacy download en_core_web_lg

In [4]:
ccars_csv = '/Users/kwheatley/Desktop/Capstone/gcloud_data/job_postings%2Fvirginia_job_posts.csv'
ccars_data = pd.read_csv(ccars_csv)

In [30]:
# Remove all job postings with null job title
ccars_data = ccars_data[~ccars_data.title.isnull()]

# Clean non words from job titles and lowercase
ccars_data.title = ccars_data.title.apply(lambda x: re.sub(r'\W+', ' ', x).lower())
ccars_data.title = ccars_data.title.apply(lambda x: re.sub(r'[0-9]+', '', x).strip())

ccars_data.head(1)

In [33]:
word_list = [] # List of all the words used
jobs_list = [] # All job titles split into tokens

for job_title in virginia_data.title:
    [word_list.append(x) for x in job_title.split()]
    jobs_list.append(job_title)

counts = Counter(word_list)

with open('ccars_dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in counts.items():
       writer.writerow([key, value])

In [40]:
# Get a list of the words to be removed
# We are going to eliminate all words with less than a certain number of occurrences
most_common_words = 300

# Words identified as needed to be retained but under the cutoff
words_to_keep =['internship', 'vice', 'president', 'ceo', 'cto', 'clo', 'cio', 'cfo', 'coo','vi',
                'vii','viii','viiv','vp','scrum','master','logistics','sales','data','extern','externship']

# Stop words will be removed from job titles
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 
              'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 
              'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 
              'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
              'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 
              'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 
              'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 
              'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 
              'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 
              'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 
              'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 
              'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 
              'how', 'further', 'was', 'here', 'than']

In [41]:
# This is a dictionary for all the available words in a job title
# Currently, we are only looking at the x most common words
WORDS = dict(counts.most_common(most_common_words))

# We have removed all stop words from the list
for word in stop_words:
    if word in WORDS:
        del WORDS[word]

# We have added back in a list of specific words
for word in words_to_keep:
    WORDS[word] = 400

In [50]:
# Get list of all words to test for misspelling
all_words = set(word_list)

[all_words.remove(word) for word in WORDS if word in all_words]

words_to_remove = []
for word in all_words:
    if len(word) < 6:
        words_to_remove.append(word)
        
for word in words_to_remove:
    all_words.remove(word)

In [51]:
# This code returns the list of misspellings
def words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    if word in WORDS.keys():
        return WORDS[word] / N
    else:
        return .0001

def corrections(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [53]:
print(datetime.datetime.now())

misspellings = []
correction = []
for word in all_words:
    corrected_word = corrections(word)
    if corrected_word != word:
        misspellings.append(word)
        correction.append(corrected_word)

dict_misspellings = list(zip(misspellings,correction))
pd.DataFrame(dict_misspellings).to_csv("ccars_misspellings_list.csv")
print(datetime.datetime.now())

2018-06-16 11:46:10.103515
2018-06-16 12:12:12.176593
